In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sys

import sys
sys.path.append("/data/jerrylee/pjt/BIGFAM.v.2.0")
from src import obj1, tools
import importlib

In [23]:
cohort = "simulation"# UKB, GS, simulation

# Step 1. Load FR-reg

In [24]:
frreg_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/frreg"
slope_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/obj1/slope-test"

In [25]:
def select_DOR_level_frreg(fns):
    fns = [fn for fn in fns if fn.split(".")[1] == "DOR"]
    return fns

def select_slope_file(fns):
    fns = [fn for fn in fns if fn.split(".")[1] != "raw"]
    return fns

In [26]:
# frreg, slope fns
frreg_fns = os.listdir(frreg_path)
frreg_fns = select_DOR_level_frreg(frreg_fns)

slope_fns = os.listdir(slope_path)
slope_fns = select_slope_file(slope_fns)
len(frreg_fns), len(slope_fns)

(7290, 7290)

In [27]:
frreg_phenos = [fn.split(".")[0] for fn in frreg_fns]
slope_phenos = [fn.split(".")[0] for fn in slope_fns]

common_phenos = [pheno for pheno in frreg_phenos if pheno in slope_phenos]
len(common_phenos)

7290

# Step 2. Prediction

In [28]:
output_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/obj1/prediction"
output_path

'/data/jerrylee/pjt/BIGFAM.v.2.0/data/simulation/obj1/prediction'

In [29]:
def get_slope_test_results(df_slope_test):
    lower, upper = df_slope_test.loc[
        df_slope_test["param"] == "slope", 
        ["lower(2.5%)", "upper(97.5%)"]
    ].values[0]
    
    return([lower, upper])

In [30]:
for ii, pheno in enumerate(tqdm(common_phenos)):
    # load FR-reg
    frreg_fn = f"{frreg_path}/{pheno}.DOR.frreg"
    df_frreg = pd.read_csv(
        frreg_fn, 
        sep='\t'
    )
    
    # resampling FR-reg to compute CIs
    df_lmbds = obj1.resampleFrregCoefficients(df_frreg, n_resample=100)    
    
    # load slope test results
    df_slope_test = pd.read_csv(
        f"{slope_path}/{pheno}.slope", 
        sep = '\t'
    )
    
    # save slope test results
    df_gsw = obj1.prediction(
        df_lmbds, 
        get_slope_test_results(df_slope_test),
        print_prog=True
    )
    
    # # save prediction results 
    savefn_pred = f"{output_path}/{pheno}"
    (tools
     .raw2long(df_gsw, params=df_gsw.columns)
     .to_csv(
         f"{savefn_pred}.pred", 
         sep='\t', 
         index=False)
     )
    df_gsw.to_csv(
        f"{savefn_pred}.raw.pred", 
         sep='\t', 
         index=False
    )
    
    break

  0%|          | 0/7290 [00:00<?, ?it/s]

1..2..3..4..5..6..7..8..9..10..

  0%|          | 0/7290 [02:59<?, ?it/s]
